In [35]:
import os

# 설정한 환경 변수 가져오기
import scraping_packers_homepage.env as env
llm = env.llm
search_engine_id = ''
api_key = ''



In [ ]:
import scraping_packers_homepage.tools.google_searcher as google_searcher

target_name = 'Sesaco Corporation'
search_result = google_searcher.search(target_name, search_engine_id, api_key
                                       , siteSearch="panjiva.com"
                                       )
search_result

In [16]:
import time
from google.cloud import bigquery
import scraping_packers_homepage.tools.bing_searcher as bing_searcher

query = """
select source_id
    , uuid
    , packer_name
    , country
    , JSON_STRIP_NULLS(json_Array(handled_sku1, handled_sku2, handled_sku3, handled_sku4, handled_sku5, handled_sku6)) as sku_list
from `greenlabs-data-farmmorning.content_analysis.gs_crawling_packer_info_raw`
where source_id = 'test_verity_20240222'
"""

bigquery_client = bigquery.Client(project="grainscanner")
query_job = bigquery_client.query(query)
query_job.result()

site_list = ['panjiva.com',
             'www.importgenius.co.kr',
             'www.exportgenius.in',
             'www.seair.co.in',
             'importkey.com']

import json
MAX_INSERT_PACKER = 2

result_list = []
for i, row in enumerate(query_job):
    
    packer_name = row.packer_name
    for site in site_list:
        query = f"site:{site} {packer_name}"
        try:
            params = { 'count': 50, 'responseFilter':'Webpages' }
            result = json.dumps(bing_searcher.search(query, '2d20c255950948d6b0ffa9685720117d', kwargs=params), ensure_ascii=False)
        except Exception as e:
            result = str(e)
        
        result_list.append({
            "source_id": row.source_id,
            "uuid": row.uuid,
            "data_type": "validation_import",
            "job_type": "search",
            "job_detail": "bing",
            "raw_data": query,
            "processed_data": result
        })
        time.sleep(0.1)
        break
    
    if (i%MAX_INSERT_PACKER) == (MAX_INSERT_PACKER-1):
        print(f' ** flush - {len(result_list)} rows')
        bigquery_client.insert_rows_json("greenlabs-data-farmmorning.content_analysis.gs_crawling_packer_info_processed", result_list)
        result_list = []
    break

if result_list:
    bigquery_client.insert_rows_json("greenlabs-data-farmmorning.content_analysis.gs_crawling_packer_info_processed", result_list)




In [17]:
result

'{"_type": "SearchResponse", "queryContext": {"originalQuery": "site:panjiva.com Montana Milling Inc"}, "webPages": {"webSearchUrl": "https://www.bing.com/search?q=site%3apanjiva.com+Montana+Milling+Inc", "totalEstimatedMatches": 13700, "value": [{"id": "https://api.bing.microsoft.com/api/v7/#WebPages.0", "name": "Manufacturers of montana panel and Suppliers of montana panel", "url": "https://panjiva.com/Manufacturers-Of/montana+panel", "isFamilyFriendly": true, "displayUrl": "https://panjiva.com/Manufacturers-Of/montana+panel", "snippet": "manufacturers and suppliers of montana panel from around the world. Panjiva uses over 30 international data sources to help you find qualified vendors of montana panel.", "dateLastCrawled": "2023-02-01T06:05:00.0000000Z", "cachedPageUrl": "http://cc.bingj.com/cache.aspx?q=site%3apanjiva.com+Montana+Milling+Inc&d=4780470760392086&mkt=ko-KR&setlang=ko-KR&w=yhGrg1--UBd96E0XW59HUX7qjbE44Wdk", "language": "en", "isNavigational": false}, {"id": "https://a

In [ ]:
import json
import os 
from pprint import pprint
import requests

'''
This sample makes a call to the Bing Web Search API with a query and returns relevant web search.
Documentation: https://docs.microsoft.com/en-us/bing/search-apis/bing-web-search/overview
'''

# Add your Bing Search V7 subscription key and endpoint to your environment variables.
subscription_key = 'key'
endpoint = "https://api.bing.microsoft.com/v7.0/search"

# Query term(s) to search for. 
#query = "panjiva.com Sesaco Corporation 78701"
query = "site:panjiva.com Sesaco Corporation"
#query = '농업회사법인 주식회사 +"해피타임 농원" +"6567" +"010"'

# Construct a request
mkt = 'en-US'
params = { 'q': query, 'mkt': mkt, 'count': 50, 'responseFilter':'Webpages' }
headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

# Call the API
try:
    response = requests.get(endpoint, headers=headers, params=params)
    response.raise_for_status()

    print("Headers:")
    print(response.headers)

    print("JSON Response:")
    pprint(response.json())
except Exception as ex:
    raise ex
    

In [104]:
target_name = 'Sesaco Corporation'

In [115]:
import scraping_packers_homepage.tools.cleanser as cleanser
import scraping_packers_homepage.tools.text_similarity as text_similarity
import importlib
importlib.reload(text_similarity)

from scraping_packers_homepage.tools.cleanser import cleansing_comp_name
from scraping_packers_homepage.tools.text_similarity import measure_text_similarity
import re, json

cleansed_target_name = cleansing_comp_name(target_name)
for item in search_result['webPages']['value']:
    title = item['name']
    comp_name = re.split(r',|\|', title)[0].strip()
    cleansed_comp_name = cleansing_comp_name(comp_name)
    similarity = measure_text_similarity(cleansed_comp_name, cleansed_target_name)
    if cleansed_comp_name.replace(' ', '') == cleansed_target_name.replace(' ', ''):
        print(item['url'])
    print(item['name'])


Sesaco Corp., 6201 EAST OLTORF ST AUSTIN TX 78741 US - Panjiva
Sesaco Corporation - Panjiva
https://panjiva.com/Sesaco-Corporation/87477611
Sesaco Corporation, E. BEN WHITE BLVD STE AUSTIN 78741 - Panjiva
Suppliers from United States | American Manufacturers — Panjiva
https://cn.panjiva.com/Sesaco-Corporation/92367206
Sesaco Corporation, S. LAMAR BLVD. STE. 580 PMB #188 AUSTIN 78745 ...
https://cn.panjiva.com/Sesaco-Corporation/32805907
Sesaco Corporation, EAST OLTORF STREET AUSTIN 78741 - 磐聚网
https://panjiva.com/Sesaco-Corporation/104188256
Sesaco Corporation, CONGRESS AVE. AUSTIN 78701 - Panjiva
https://panjiva.com/Sesaco-Corporation/32805907
Sesaco Corporation, EAST OLTORF STREET AUSTIN 78741 - Panjiva
Suppliers from United States | American Manufacturers — Panjiva
Suppliers from United States | American Manufacturers — Panjiva
Suppliers from United States | American Manufacturers — Panjiva
https://panjiva.com/Sesaco-Corporation/135416582
Sesaco Corporation, 600 CONGRESS AVE. FLOOR 

In [116]:
import requests
from bs4 import BeautifulSoup

url = 'https://panjiva.com/Sesaco-Corporation/87477611'
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}
page_response = requests.get(url, headers)
soup = BeautifulSoup(page_response.text, 'html.parser')

soup.find('h1').find_next_sibling('span').find_next_sibling('span').text.strip()

In [128]:
soup.find('a', class_="itemHtsCode").text.strip().replace('\xa0', ' ')

'HS 12 - Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder'

In [124]:
query = """
SELECT p.id, p.name, array_agg(distinct cc.name ignore nulls) as country, array_agg(distinct sku.name_en ignore nulls) as sku
FROM `greenlabs-data-farmmorning.stream_ods_grainscanner.packer` as p
inner join `greenlabs-data-farmmorning.stream_ods_grainscanner.packer_country` as pc
  on p.id = pc.packer_id
inner join `greenlabs-data-farmmorning.stream_ods_grainscanner.common_country` as cc
  on pc.country_id = cc.id
left outer join `greenlabs-data-farmmorning.stream_ods_grainscanner.packer_sku` as ps
  on p.id = ps.packer_id
left outer join `greenlabs-data-farmmorning.stream_ods_grainscanner.sku` as sku
  on ps.sku_id = sku.id
WHERE p.is_deleted = 0 
  and p.contact_stage = 'PCS01' 
  and p.verification_stage = 'PVS01'
group by 1, 2
order by 1, 2
"""

'United States'

In [2]:

#Copyright (c) Microsoft Corporation. All rights reserved.
#Licensed under the MIT License.

# -*- coding: utf-8 -*-

import json
import os 
from pprint import pprint
import requests

'''
This sample makes a call to the Bing Web Search API with a query and returns relevant web search.
Documentation: https://docs.microsoft.com/en-us/bing/search-apis/bing-web-search/overview
'''

# Add your Bing Search V7 subscription key and endpoint to your environment variables.
subscription_key = "key"
endpoint = "https://api.bing.microsoft.com/v7.0/search"


# Query term(s) to search for. 
query = "Microsoft"

# Construct a request
mkt = 'en-US'
params = { 'q': query, 'mkt': mkt }
headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

# Call the API
try:
    response = requests.get(endpoint, headers=headers, params=params)
    response.raise_for_status()

    print("Headers:")
    print(response.headers)

    print("JSON Response:")
    pprint(response.json())
except Exception as ex:
    raise ex
    

Headers:
{'Cache-Control': 'private, max-age=0', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'Content-Encoding': 'gzip', 'Expires': 'Wed, 06 Mar 2024 01:00:50 GMT', 'Vary': 'Accept-Encoding', 'P3P': 'CP="NON UNI COM NAV STA LOC CURa DEVa PSAa PSDa OUR IND"', 'X-EventID': '65e7c07e9ff54c06bf57fa83a7c718bf', 'UserAgentReductionOptOut': 'A7kgTC5xdZ2WIVGZEfb1hUoNuvjzOZX3VIV/BA6C18kQOOF50Q0D3oWoAm49k3BQImkujKILc7JmPysWk3CSjwUAAACMeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiU2VuZEZ1bGxVc2VyQWdlbnRBZnRlclJlZHVjdGlvbiIsImV4cGlyeSI6MTY4NDg4NjM5OSwiaXNTdWJkb21haW4iOnRydWUsImlzVGhpcmRQYXJ0eSI6dHJ1ZX0=', 'BingAPIs-TraceId': '65e7c07e9ff54c06bf57fa83a7c718bf', 'BingAPIs-SessionId': '8C8262D3F8D5443A89B98B0636375E9E', 'X-MSEdge-ClientID': '3345136491396BB90ADF075E900C6A8C', 'X-MSAPI-UserState': 'ce28', 'BingAPIs-Market': 'en-US', 'BingAPIs-RGUID': 'ed01acdfe3a24fde8afa36e840444395', 'X-Search-ResponseInfo': 'InternalResponseTime=474,MSDatace